In [1]:
import pandas as pd
import sqlite3
import time
import numpy as np

path = '/home/td/Documents'


In [2]:
url = 'https://s3-us-west-2.amazonaws.com/pcadsassessment/parking_citations.corrupted.csv'
df = pd.read_csv(url)
df.to_csv('{path}/tickets.csv'.format(path = path), index = False)
    

/home/td/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
def pandas_most_common_makes(df):
    return df['Make'].value_counts()[:25].index.tolist()


def sqlite_most_common_names(conn):
    res = conn.execute('''Select Make, COUNT(Make) As count_col
                    from tickets
                    group by Make
                    order by count_col DESC
                    LIMIT 25;''')
    
    res = [i[0] for i in res]
    print('here', res)
    return res


def pandas_most_common_color_per_make(df):
    result_series =  df.groupby('Make')['Color'].agg(lambda x: x.value_counts(dropna = False).index[0])
    return result_series.to_dict()


def sqlites_most_common_color_per_make(conn):
    res = conn.execute('''SELECT FinalTable.Make, FinalTable.Color    
                            FROM    
                             ((SELECT tickets.Make, tickets.Color, Count(tickets.Color) AS color_count
                              FROM tickets
                              GROUP BY tickets.Make, tickets.Color) As CountMakeColor
                            JOIN
                             (SELECT dT.Make, Max(dT.color_count) As max_color_count
                              FROM
                                   (SELECT tickets.Make, tickets.Color, Count(tickets.Color) AS color_count
                                    FROM tickets
                                    GROUP BY tickets.Make, tickets.Color) As dT
                              GROUP BY dT.Make) As MaxColorCount
                              
                        ON CountMakeColor.Make = MaxColorCount.Make 
                        AND CountMakeColor.color_count = MaxColorCount.max_color_count) as FinalTable''')

    res_dict = dict()
    for i in res:
        res_dict.setdefault(i[0], [])
        res_dict[i[0]].append(i[1])
    for i in res_dict:
        res_dict[i].sort()
    
    return {i:j[0] for i, j in res_dict.items()}


def pandas_first_ticket_per_make(df):
    df_sorted = df.sort_values(by = ['Issue Date', "Issue time"])
    result_series = df_sorted.groupby('Make')['Ticket number'].agg(lambda x: x.tolist()[0])
    return result_series.to_dict()


def sqlite_first_ticket_per_make(conn):
    #     res = conn.execute('''SELECT FinalTable.Make, FinalTable."Ticket number"    
    #                             FROM    
    #                              ((SELECT tickets.Make, MIN(tickets."Issue Date" ||substr(tickets."Issue time", -10, 10)) as ts1
    #                                 FROM tickets
    #                                  GROUP BY tickets.Make) As makes_earliest_ticket_date
    #                             JOIN
    #                              (SELECT tickets.Make, "Ticket number", tickets."Issue Date" ||substr(tickets."Issue time", -10, 10) as ts2
    #                                     FROM tickets) As all_dated_tickets

    #                         ON makes_earliest_ticket_date.Make = all_dated_tickets.Make 
    #                         AND makes_earliest_ticket_date.ts1 = all_dated_tickets.ts2) as FinalTable ''')
    res = conn.execute('''select Make, "Ticket number"
                            from tickets JOIN 
                            (select Make as group_make, min("Issue Date") as earliest_date 
                            from tickets 
                            group by group_make) 
                            on group_make = make and earliest_date = "Issue Date";''')
    res_dict = dict()
    for i in res:
        res_dict[i[0]] = i[1]
    
    return res_dict

In [13]:
# def test_f(conn):
#     res = conn.execute('''SELECT FinalTable.Make, FinalTable."Ticket number"    
#                             FROM    
#                              ((SELECT tickets.Make, MIN(tickets."Issue Date" ||substr(tickets."Issue time", -10, 10)) as ts1
#                                 FROM tickets
#                                  GROUP BY tickets.Make) As makes_earliest_ticket_date
#                             JOIN
#                              (SELECT tickets.Make, "Ticket number", tickets."Issue Date" ||substr(tickets."Issue time", -10, 10) as ts2
#                                     FROM tickets
#                                      GROUP BY tickets.Make) As all_dated_tickets
                              
#                         ON makes_earliest_ticket_date.Make = all_dated_tickets.Make 
#                         AND makes_earliest_ticket_date.ts1 = all_dated_tickets.ts2) as FinalTable ''')
#     res2 =  conn.execute('''SELECT tickets.Make, MIN(tickets."Issue Date" ||substr(tickets."Issue time", -10, 10)) as ts1
#                                 FROM tickets
#                                  GROUP BY tickets.Make''')
#     res3 =  conn.execute('''SELECT tickets.Make, MIN(tickets."Issue Date" ||substr(tickets."Issue time", -10, 10)) as ts1
#                                 FROM tickets
#                                  GROUP BY tickets.Make''')
#     for i in res:
#         print('res', i)
#     for i in res2

In [14]:

df_copy = df.sample(n = 1000)
with sqlite3.connect(':memory:') as conn_mem:
    df_copy.to_sql('tickets', conn_mem, if_exists='replace')
    sqlite_first_ticket_per_make(conn_mem)

In [15]:
def run_timings(df, n):
    if n:
        df = df.sample(n=n)
        
    results = dict()
    results['number_of_records'] = df.shape[0]
    
    
    with sqlite3.connect(':memory:') as conn_mem, sqlite3.connect('tickets.db') as conn_disk:
        df.to_sql('tickets', conn_mem, if_exists='replace')
        df.to_sql('tickets', conn_disk, if_exists='replace')
        
        pandas_q1_start = time.time()
        res_pd = pandas_most_common_makes(df)
        pandas_q1_end = time.time()
        
        sql_q1_disk_start = time.time()
        res_sql_disk = sqlite_most_common_names(conn_disk)
        sql_q1_disk_end = time.time()
        
        sql_q1_mem_start = time.time()
        res_sql_mem = sqlite_most_common_names(conn_mem)
        sql_q1_mem_end = time.time()
        
        results['q1_pandas_time'] = pandas_q1_end - pandas_q1_start
        results['q1_sql_disk_time'] = sql_q1_disk_end - sql_q1_disk_start
        results['q1_sql_mem_time'] = sql_q1_mem_end - sql_q1_mem_start
        print(results)
        
        try:
            assert sorted(res_pd) == sorted(res_sql_disk)
        except:
            print(sorted(res_pd))
            print(sorted(res_sql_mem))
            print('difference between pandas and sql, could still be valid in cases of ties: {}'.format(set(res_pd) ^ set(res_sql_disk)))

            
        # q2
        print()
        print('q2')
        pandas_q2_start = time.time()
        res_pd = pandas_most_common_color_per_make(df)
        pandas_q2_end = time.time()
        print(pandas_q2_end - pandas_q2_start)

        sql_q2_disk_start = time.time()
        res_sql_disk = sqlites_most_common_color_per_make(conn_disk)
        sql_q2_disk_end = time.time()
        print(sql_q2_disk_end - sql_q2_disk_start)

        sql_q2_mem_start = time.time()
        res_sql_mem = sqlites_most_common_color_per_make(conn_mem)
        sql_q2_mem_end = time.time()
        print(sql_q2_mem_end - sql_q2_mem_start)
        
        results['q2_pandas_time'] = pandas_q2_end - pandas_q2_start
        results['q2_sql_disk_time'] = sql_q2_disk_end - sql_q2_disk_start
        results['q2_sql_mem_time'] = sql_q2_mem_end - sql_q2_mem_start
        
        
        try:
            assert res_pd == res_sql_disk
        except:
            set1 = set(res_pd.items())
            set2 = set(res_sql_disk.items())
            print(set1 ^ set2)
            print('difference between pandas and sql, could still be valid in cases of ties: {}'.format(set1 ^ set2))
            

        # q3
        print()
        print('q3')
        pandas_q3_start = time.time()
        res_pd = pandas_first_ticket_per_make(df)
        pandas_q3_end = time.time()
        print(pandas_q2_end - pandas_q2_start)

        sql_q3_disk_start = time.time()
        res_sql_disk = sqlite_first_ticket_per_make(conn_disk)
        sql_q3_disk_end = time.time()
        print(sql_q3_disk_end - sql_q3_disk_start)

        sql_q3_mem_start = time.time()
        res_sql_mem = sqlite_first_ticket_per_make(conn_mem)
        sql_q3_mem_end = time.time()
        print(sql_q3_mem_end - sql_q3_mem_start)
        
        results['q3_pandas_time'] = pandas_q3_end - pandas_q3_start
        results['q3_sql_disk_time'] = sql_q3_disk_end - sql_q3_disk_start
        results['q3_sql_mem_time'] = sql_q3_mem_end - sql_q3_mem_start

        try:
            assert sorted(res_pd) == sorted(res_sql_disk)
            assert sorted(res_sql_disk) == sorted(res_sql_mem)
        except:
            print(res_pd)
            print(res_sql_disk)
            set1 = set(res_pd.items())
            set2 = set(res_sql_disk.items())
            print(set1 ^ set2)
        
        return [results]
            
            
        

In [16]:
results = []

df_with_make = df.dropna(subset= ['Make'])

for i in [1000, 2000, 4000, 8000, 16000, 32000, 64000, 128000, 256000, None]:
    results.extend(run_timings(df_with_make, i))
    
res_df = pd.DataFrame.from_dict(results)

    

here ['TOYT', 'HOND', 'FORD', 'NISS', 'CHEV', 'BMW', 'VOLK', 'MERZ', 'DODG', 'LEXS', 'KIA', 'OTHR', 'HYUN', 'JEEP', 'GMC', 'AUDI', 'MAZD', 'SUBA', 'ACUR', 'CADI', 'CHRY', 'INFI', 'VOLV', 'TOYO', 'JAGU']
here ['TOYT', 'HOND', 'FORD', 'NISS', 'CHEV', 'BMW', 'VOLK', 'MERZ', 'DODG', 'LEXS', 'KIA', 'OTHR', 'HYUN', 'JEEP', 'GMC', 'AUDI', 'MAZD', 'SUBA', 'ACUR', 'CADI', 'CHRY', 'INFI', 'VOLV', 'TOYO', 'JAGU']
{'number_of_records': 1000, 'q1_pandas_time': 0.0010478496551513672, 'q1_sql_disk_time': 0.00043964385986328125, 'q1_sql_mem_time': 0.0006313323974609375}

q2
0.026201486587524414
0.000972747802734375
0.0009405612945556641
{('SUBA', 'SL'), ('JAGU', 'BK'), ('SUBA', 'BK'), ('PONT', 'GY'), ('JEEP', 'WT'), ('JAGU', 'GY'), ('JEEP', 'BK'), ('PONT', 'PR')}
difference between pandas and sql, could still be valid in cases of ties: {('SUBA', 'SL'), ('JAGU', 'BK'), ('SUBA', 'BK'), ('PONT', 'GY'), ('JEEP', 'WT'), ('JAGU', 'GY'), ('JEEP', 'BK'), ('PONT', 'PR')}

q3
0.026201486587524414
0.000516891479

0.13759207725524902
0.13351964950561523
{('PACK', 'BN'), ('PEUG', 'BG'), ('IND', 'BK'), ('PACK', 'BK'), ('TESL', 'BK'), ('MERK', 'BK'), ('KAWA', 'RE'), ('STRI', 'OR'), ('STRI', 'BK'), ('JAG', 'GY'), ('PEUG', 'BK'), ('KAWA', 'BL'), ('JAG', 'BL'), ('IND', 'GY'), ('TESL', 'GY'), ('MERK', 'GY')}
difference between pandas and sql, could still be valid in cases of ties: {('PACK', 'BN'), ('PEUG', 'BG'), ('IND', 'BK'), ('PACK', 'BK'), ('TESL', 'BK'), ('MERK', 'BK'), ('KAWA', 'RE'), ('STRI', 'OR'), ('STRI', 'BK'), ('JAG', 'GY'), ('PEUG', 'BK'), ('KAWA', 'BL'), ('JAG', 'BL'), ('IND', 'GY'), ('TESL', 'GY'), ('MERK', 'GY')}

q3
0.13277387619018555
0.0785365104675293
0.07601332664489746
here ['TOYT', 'HOND', 'FORD', 'NISS', 'CHEV', 'BMW', 'MERZ', 'VOLK', 'HYUN', 'DODG', 'LEXS', 'KIA', 'JEEP', 'AUDI', 'MAZD', 'OTHR', 'GMC', 'INFI', 'CHRY', 'ACUR', 'SUBA', 'VOLV', 'TOYO', 'MITS', 'CADI']
here ['TOYT', 'HOND', 'FORD', 'NISS', 'CHEV', 'BMW', 'MERZ', 'VOLK', 'HYUN', 'DODG', 'LEXS', 'KIA', 'JEEP', 'AUDI'

In [17]:
res_df['q1_sql_speed_difference'] = res_df['q1_sql_mem_time']/res_df['q1_pandas_time']
res_df['q2_sql_speed_difference'] = res_df['q2_sql_mem_time']/res_df['q2_pandas_time']
res_df['q3_sql_speed_difference'] = res_df['q3_sql_mem_time']/res_df['q3_pandas_time']

In [18]:
res_df

,number_of_records,q1_pandas_time,q1_sql_disk_time,q1_sql_mem_time,q2_pandas_time,q2_sql_disk_time,q2_sql_mem_time,q3_pandas_time,q3_sql_disk_time,q3_sql_mem_time,q1_sql_speed_difference,q2_sql_speed_difference,q3_sql_speed_difference
0,1000,0.001048,0.000440,0.000631,0.026201,0.000973,0.000941,0.003222,0.000517,0.000490,0.602503,0.035897,0.152087
1,2000,0.001080,0.000607,0.000707,0.028392,0.001743,0.001689,0.004145,0.000988,0.000966,0.654305,0.059480,0.233105
2,4000,0.001073,0.000982,0.001155,0.033843,0.003177,0.003203,0.005064,0.001894,0.001841,1.075983,0.094641,0.363589
3,8000,0.002787,0.004024,0.003885,0.047818,0.006566,0.006514,0.007565,0.003954,0.003861,1.393960,0.136226,0.510416
4,16000,0.001747,0.004026,0.004008,0.049637,0.014020,0.013599,0.011865,0.008283,0.007896,2.294567,0.273961,0.665474
5,32000,0.002602,0.008381,0.008020,0.059836,0.028904,0.028222,0.020025,0.016699,0.016040,3.082287,0.471664,0.801024
6,64000,0.004330,0.017276,0.016884,0.092326,0.061122,0.059433,0.036413,0.037938,0.036877,3.899455,0.643727,1.012748
7,128000,0.007709,0.038390,0.037416,0.132774,0.137592,0.133520,0.067545,0.078537,0.076013,4.853467,1.005617,1.125378
8,256000,0.016214,0.078674,0.076442,0.183422,0.284568,0.278690,0.130197,0.156088,0.153938,4.714500,1.519391,1.182347
9,4357544,0.186349,1.522065,1.472521,1.083089,5.967988,5.870193,1.507195,2.960209,2.877785,7.901954,5.419864,1.909364


The sql for part 3 is not entirely correct, it will get a record from the first ticket day but not nessesarily the earliest time.

My metrics show that my pandas code is faster at large data sizes and scales better. I find the pandas much simpler to write and understand, it is objectively much shorter.

I was surprized that sqlite was slower, it is possible that my sql code is not optimal but the benchmarks linked below show that pandas can outperform sqlite for joining and grouping operations. My code uses both. SQLite may be a better choice for simple data filtering and selection.

https://blog.thedataincubator.com/2018/05/sqlite-vs-pandas-performance-benchmarks/


In [19]:
df.head()

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,1103341116,2015-12-21T00:00:00,1251.0,NaN,NaN,CA,200304.0,NaN,NaN,PA,GY,13147 WELBY WAY,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0
1,1103700150,2015-12-21T00:00:00,1435.0,NaN,NaN,CA,201512.0,NaN,NaN,VN,WH,525 S MAIN ST,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0
2,1104803000,2015-12-21T00:00:00,2055.0,NaN,NaN,CA,201503.0,NaN,NaN,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE CURB,58.0,6439997.9,1802686.4
3,1104820732,2015-12-26T00:00:00,1515.0,NaN,NaN,CA,NaN,NaN,NaN,PA,WH,100 WORLD WAY,2F11,2.0,000,17104h,NaN,6440041.1,1802686.2
4,1105461453,2015-09-15T00:00:00,115.0,NaN,NaN,CA,200316.0,NaN,CHEV,PA,BK,GEORGIA ST/OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,99999.0,99999.0


In [20]:
df['ticket_dt'] = pd.to_datetime(df['Issue Date'], errors='coerce')

df['plate_expiration_dt'] = pd.to_datetime(df['Plate Expiry Date'].astype(int, errors = 'ignore'), format = '%Y%m', errors='coerce')
df['is_expired_at_time_of_ticket'] = np.nan
df.loc[df['ticket_dt'] > df['plate_expiration_dt'], 'is_expired_at_time_of_ticket'] = 1
df.loc[df['ticket_dt'] <= df['plate_expiration_dt'], 'is_expired_at_time_of_ticket'] = 0
df.loc[df['RP State Plate'] != 'CA', 'in_state'] = 0
df.loc[df['RP State Plate'] == 'CA', 'in_state'] = 1

df_valid_dates = df.dropna(subset = ['is_expired_at_time_of_ticket'])

out_of_state_expired_perc =  df_valid_dates[df_valid_dates['in_state'] == 0]['is_expired_at_time_of_ticket'].mean()
in_state_expired_perc =  df_valid_dates[df_valid_dates['in_state'] == 1]['is_expired_at_time_of_ticket'].mean()
from statsmodels.stats.weightstats import ztest

t_stat, p_value = ztest(df_valid_dates[df_valid_dates['in_state'] == 0]['is_expired_at_time_of_ticket'], 
                        df_valid_dates[df_valid_dates['in_state'] == 1]['is_expired_at_time_of_ticket'])

print('out of state perc expired: {0}'.format(out_of_state_expired_perc))
print('in state perc expired: {0}'.format(in_state_expired_perc))
print('one tailed z test p value: {0}'.format(p_value))

out of state perc expired: 0.23402867577376638
in state perc expired: 0.2200086962308552
one tailed z test p value: 9.059329489508932e-105


In [21]:
df_valid_dates.head()

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,...,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude,ticket_dt,plate_expiration_dt,is_expired_at_time_of_ticket,in_state
0,1103341116,2015-12-21T00:00:00,1251.0,NaN,NaN,CA,200304.0,NaN,NaN,PA,...,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0,2015-12-21,2003-04-01,1.0,1.0
1,1103700150,2015-12-21T00:00:00,1435.0,NaN,NaN,CA,201512.0,NaN,NaN,VN,...,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0,2015-12-21,2015-12-01,1.0,1.0
2,1104803000,2015-12-21T00:00:00,2055.0,NaN,NaN,CA,201503.0,NaN,NaN,PA,...,2.0,8939,WHITE CURB,58.0,6439997.9,1802686.4,2015-12-21,2015-03-01,1.0,1.0
5,1106226590,2015-09-15T00:00:00,19.0,NaN,NaN,CA,201507.0,NaN,NaN,VN,...,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0,2015-09-15,2015-07-01,1.0,1.0
6,1106500452,2015-12-17T00:00:00,1710.0,NaN,NaN,CA,201605.0,NaN,MAZD,PA,...,1.0,8070,PARK IN GRID LOCK ZN,163.0,99999.0,99999.0,2015-12-17,2016-05-01,0.0,1.0
